# Self-Attention

Input Layerでつくったパッチをよりよいパッチにする。

In [2]:
# self-attention の説明を読んで、雰囲気の例。Self-Attention自体を表すものではない。
from random import seed, randint


class Human:
    def __init__(self, height, weight, age):
        self.height = height
        self.weight = weight
        self.age = age

seed(0)
a = Human(randint(160, 200), randint(60, 80), randint(20, 80))
b = Human(randint(160, 200), randint(60, 80), randint(20, 80))
c = Human(randint(160, 200), randint(60, 80), randint(20, 80))
d = Human(randint(160, 200), randint(60, 80), randint(20, 80))

# a と a, b, c, dがどれくらい似ているかを数値化する
# 特徴量 = 各Humanごとの慎重・体重・年齢を足した値
# 類似度 = そのHumanの特徴量 / aの特徴量

features = [ sum([h.__dict__[k] for k in h.__dict__]) for h in (a, b, c, d) ]

# 乱数を固定しているので、feature = [279, 303, 304, 337]
# aに一番近いのは a でその次に　aに近いのは b。

# 一番近いaを60%、二番目近いbを30%、あとの二つを5%の重みづけでaを更新する

new_a = Human(
    weight=0.6*a.weight+0.3*b.weight+0.05*(c.weight + d.weight),
    height=0.6*a.height+0.3*b.height+0.05*(c.height + d.height),
    age=0.6*a.age+0.3*b.age+0.05*(c.age + d.age),
)

new_a.__dict__


{'height': 181.54999999999998, 'weight': 73.94999999999999, 'age': 34.85}

## 流れ

1. 埋め込み
   1. Input Layerのベクトルを別々の線形層でqkv(Query, Key, Value)を作成
2. 内積
   1. q, kの行列積$qk^T$を計算
      1. $q^Tk$ではなく$qt^K$なのは、行方向にデータを保持しているから
   2. $qk^T$にSoftMax関数を適用して正規化する
   3. $A = softmax(\frac{qk^T}{\sqrt{D_h}})$ をAttention Weightという
3. の加重和
   1. $A$にvを加重和
   2. $Av$をScaled Dot-Product Attention という
4. MHSA(Multi-Head Self-Attention)
   1. 「1.埋め込み」で作成したq,k,vをヘッド数?の数に分割する
   2. 分割したq,k,vに内積して、複数のAttention Weightを作成する
   3. 各Attention Weightと分割したと各vを加重和して、複数のScaled Dot-Prodct Attentionを作成する


## 数式表現

### 1. 埋め込み
> $z \in \R^{N \times D}$

をSelf-Attentionの入力とする（$z_0$と同じ大きさ）。

$z$をq,k,vに埋め込むときのベクトルの長さを$D_h$とするととしてq,k,vの線形層はそれぞれ$W^q, W^k, W^v \in \mathbb{R}^{D \times D_h}$とすれば、

> $q = z W^q \in \mathbb{R}^{N \times D_h}$
> 
> $k = z W^q \in \mathbb{R}^{N \times D_h}$
> 
> $v = z W^q \in \mathbb{R}^{N \times D_h}$

と表せる。

### 2. 内積

$q$と$k$の行列積 $ qk^T \in \mathbb{R}^{N \times N}$ に SoftMax関数を適用する。

$$A = softmax(\frac{qk^T}{\sqrt{D_h}}) \in \mathbb{R}^{N \times N}$$


## 3. 加重和

$A$に$v$を加重和して、Scaled Dot-Product Attentionを作成する。

$$SA = Av = softmax(\frac{qk^T}{\sqrt{D_h}}) v \in \mathbb{R}^{N \times D_h}$$


## MHSA

1-3 と同様の操作をして、Scaled Dot-Product Attention を作成する。

$q = W^q z,k = W^k z,v = W^v z$ を作成するところは同じ。

少しだけ復習すると、$z$の形は次のような行列である。

| | $vec1$ |  $vec2$ | $\dots$ | $vecD$ |
| --- | --- | --- | --- | --- |
| CLS | $cls1$ | $cls2$  | $\dots$ | $clsD$ |
| $patch_1$ | $pat_{1}1$ | $pat_{1}2$ | $\dots$ | $pat_{1}D$ |
| $patch_2$ | $pat_{2}1$ | $pat_{2}2$ | $\dots$ | $pat_{2}D$ |
|  $\vdots$ | $\vdots$ | $\vdots$ | $\vdots$ | $\vdots$ 
| $patch_{N}$ | $pat_{N}1$ | $pat_{N}2$ | $\dots$ | $pat_{N}D$

CLSと$N$個のパッチが行ベクトルで並んでいる。
これらを線形層$W^q, W^k, W^v$で$q,k,v$で埋め込み、$D_h$次にする。

$q,k,v$はすべて同じ形をしているため、$q$の場合だけを明記する。

| | $q1$ |  $q2$ | $\dots$ | $q(D_h)$ |
| --- | --- | --- | --- | --- |
| CLS | $cls1$ | $cls2$  | $\dots$ | $clsD_h$ |
| $q_1$ | $q_{1}1$ | $q_{1}2$ | $\dots$ | $q_{1}D_h$ |
| $q_2$ | $q_{2}1$ | $q_{2}2$ | $\dots$ | $q_{2}D_h$ |
|  $\vdots$ | $\vdots$ | $\vdots$ | $\vdots$ | $\vdots$ 
| $q_{N}$ | $q_{N}1$ | $q_{N}2$ | $\dots$ | $q_{N}D_h$

これを **列** で $K$個に分割する。すなわち、その$i$番目の分割は次のようになる。




$q, k, v$を$K$個に分割する。

$vec(i1)$ | $vec(i2)$  | $\dots$ | $vec(i(D_h / K))$  
  ---     |   ---      |  ---  |  ---  
$cls(i1)$ | $cls(i2)$  | $\dots$ | $cls(i(D_h / K))$  
$pat_1(i1)$ | $pat_1(i2)$  | $\dots$ | $pat_1(i(D_h / K))$  
$pat_2(i1)$ | $pat_2(i2)$  | $\dots$ | $pat_2(i(D_h / K))$  
 $\vdots$ |  $\vdots$ |  $\vdots$ |  $\vdots$ 
$pat_{N}(i1)$ | $pat_{N}(i2)$  | $\dots$ | $pat_{N}(i(D_h / K))$  

元のパッチ埋め込みの次数が$D_h$でそれを$K$個に分割しているため、分割した一つは$D_h / K$個になる。

$q, k, v$をそれぞれ$K$個に分割すると

> $[q_1, q_2, \dots, q_K], \quad q_i \in  \mathbb{R}^{N \times \frac{D_h}{K}} $
> 
> $[k_1, k_2, \dots, k_K], \quad k_i \in  \mathbb{R}^{N \times \frac{D_h}{K}} $
> 
> $[q_1, q_2, \dots, q_K], \quad q_i \in  \mathbb{R}^{N \times \frac{D_h}{K}} $

$i$ 番目の$q_i$と$k_i$の内積をとると、$q_i, k_i \in \mathbb{R}^{N \times \frac{D}{K}}$ であるから、

$$q_i k_i^T \in \mathbb{R}^{N \times N}$$

と、**通常のSelf-Attentionと同じ大きさである**。これにsoftmax関数を適用して正規化する。

$$A_i = softmax(\frac{q_i k_i^T}{\sqrt{D_h} }), \quad i = 1, 2, \dots, K$$

これらを加重和して、Scaled Dot-Product Attention を作成する。

$$A_i v_i \in \mathbb{R}^{N \times D_h}, \quad i = 1, 2, \dots, K $$

これらをまとめて処理する関数を$SA_i(z)$ とすると、

$$
\begin{array}{ccc}
SA_i(z): \mathbb{R}^{N \times D} & \stackrel{f}{\longrightarrow} & \mathbb{R}^{N \times N} \\
\end{array}
$$

であり、

$$
SA_i(z) = sofmax(\frac{q_i k_i^T}{ \sqrt{D_h} } ) v_i
$$

である。

